In [29]:
# Importação das biblíotecas relevantes para o projeto
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

# Módulo 1

## 1. Aquisição e leitura de dados reais

> O DATASUS disponibiliza informações que podem servir para subsidiar  análises objetivas da situação sanitária, tomadas de decisão baseadas em evidências e elaboração de programas de ações de saúde.<br>
Link para o [**DATASUS**](https://datasus.saude.gov.br/informacoes-de-saude-tabnet/)

### 1.1 Importação do *valor total dos procedimentos hospitalares do SUS por UF e mês de **processamento***

**Informações da extração:**
- [**Informações de Saúde(TABNET)**](http://www2.datasus.gov.br/DATASUS/index.php?area=0202&id=11633&VObj=http://tabnet.datasus.gov.br/cgi/deftohtm.exe?sih/cnv/qi)
- **Localização:** Informações de Saúde(TABNET)/Assistência à Saúde/Dados Consolidados AIH (RD), por local de internação, a partir de 2008/Brasil por região e unidade de Federação
- **Data:** 18-05-2021 22:35
- **Linha:** Unidade da Federação
- **Coluna:** Ano Mês processamento
- **Conteúdo:** Valor Total
- **Período:** *de:* Jan-2008 *até:* Mar-2021
- **Nome do arquivo:** /dados/A223850189_28_143_208(proc).csv

In [296]:
# Abre e Lê o arquivo csv
df_proc = pd.read_csv('https://raw.githubusercontent.com/alura-cursos/agendamento-hospitalar/main/dados/A151346189_28_143_208.csv',
                      sep=';', decimal=',', thousands='.', engine='python', encoding='ISO-8859-1', skiprows=3, skipfooter=12)

df_proc.head()

,Unidade da Federação,2008/Jan,2008/Fev,2008/Mar,2008/Abr,2008/Mai,2008/Jun,2008/Jul,2008/Ago,2008/Set,...,2019/Dez,2020/Jan,2020/Fev,2020/Mar,2020/Abr,2020/Mai,2020/Jun,2020/Jul,2020/Ago,Total
0,11 Rondônia,1388528.39,2931283.42,1541682.52,1525314.96,1645953.84,1406150.68,3065279.01,3231494.61,3117178.63,...,8085025.66,9221412.18,7493310.88,8092260.11,7273843.63,7652120.28,8725530.68,11152107.08,10285455.78,9.291297e+08
1,12 Acre,902416.00,1497206.26,1794028.48,1730469.42,1819443.92,1828496.00,2511754.59,2089107.14,2275708.53,...,2874871.28,3331803.55,3417691.07,3076003.20,2286642.60,2896340.24,3045245.80,3915193.20,-,4.206680e+08
2,13 Amazonas,4735529.42,7118990.57,8196635.49,8259378.42,7831399.90,8477273.62,9368858.72,9352532.70,9363099.35,...,14346553.60,16018459.07,14913745.37,15068304.35,13509579.29,19658585.46,20897542.93,19346450.18,14726443.67,1.771655e+09
3,14 Roraima,657889.53,777939.31,718688.03,839994.39,862347.96,832440.66,996693.09,894271.18,910424.17,...,3097113.21,3059823.80,2912519.98,2937346.43,2868820.96,2752228.26,3401864.29,3015488.30,2826486.18,3.050109e+08
4,15 Pará,18864744.11,19553758.20,21937342.70,20842829.69,23249952.88,23240687.56,24002223.56,23341218.03,25172261.32,...,35925899.01,34345207.63,37384336.94,35512568.15,28217249.66,32617541.05,41625331.28,38438873.79,39911193.02,4.437116e+09


## 2. Processamento inicial do DF de Processamento

### 2.1 Atualização dos nomes das colunas e da coluna chamada Unidade da Federação

In [297]:
# Altera '/' por '_' no nome da coluna
df_proc.columns = df_proc.columns.str.replace('/', '-')

In [298]:
# Altera coluna "Unidades da Federação" com espaços e maiúsculas por uf
df_proc = df_proc.rename(columns={'Unidade da Federação':'uf'})

In [299]:
# Subset da coluna "uf" e divisão do código [11 Rondônia] em [11] [Rondônia]
df_proc['uf'] = df_proc['uf'].str.split(' ',n=1, expand=True)[0].astype('str')

In [300]:
# Remove coluna Total
df_proc = df_proc.drop('Total', axis=1)

### 2.2 Dataframe Número UF x Código UF (Unidade Federativa)
Em vez de utilizar o nome da unidade federativa como 'Ceará' (que tem acento) ou então 'Rio Grande do Norte' (nome muito grande), eu decidi utilizar o código UF.<br>
- A lista de referencia utilizada se encontra neste [LINK](https://www.oobj.com.br/bc/article/quais-os-c%C3%B3digos-de-cada-uf-no-brasil-465.html)
- o dicionário foi criado na mão.

In [301]:
# Cria lista na mão dos códigos UF e os UFs
uf_number_code = pd.DataFrame({"uf_number":["11", "12", "13", "14", "15", "16", "17", "21", "22", "23", "24", "25", "26", "27",
                                          "28", "29", "31", "32", "33", "35", "41", "42", "43", "50", "51", "52", "53"],
                               "uf":["RO", "AC", "AM", "RR", "PA", "AP", "TO", "MA", "PI", "CE", "RN", "PB", "PE", "AL",
                                       "SE", "BA", "MG", "ES", "RJ", "SP", "PR", "SC", "RS", "MS", "MT", "GO", "DF"]})

### 2.3 Substituição do número federativo pelo código federativo

In [302]:
# Substitui na coluna 'uf' 
df_proc['uf'] = df_proc['uf'].replace(uf_number_code['uf_number'].array,uf_number_code['uf'].array)

### 2.4 Transformação para Tidy
Os dados formatados como Tidy são mais entendiveis, e para isso, a transposição ajuda. Devemos ter:
- Cada coluna contendo uma variável
- Cada linha uma observação
- Cada célula um valor

In [303]:
# Transposição de linhas por colunas ajuda a manter o dato "TYDY", onde cada coluna é uma variável, cada linha uma observação e cada célula um valor.
df_proc = df_proc.set_index('uf').T

In [304]:
df_proc = df_proc.reset_index()
df_proc = df_proc.rename(columns={'index':'date'})
df_proc['date'] = pd.to_datetime(df_proc['date'].replace(['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez'],
                                       ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], regex=True), format='%Y-%b')
df_proc.reindex(df_proc['date'].dt.strftime('%Y-%m'))
#df_proc.columns = df_proc.columns.get_level_values(0)

uf,date,RO,AC,AM,RR,PA,AP,TO,MA,PI,...,ES,RJ,SP,PR,SC,RS,MS,MT,GO,DF
date,,,,,,,,,,,,,,,,,,,,,
2008-01,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-02,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-03,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-04,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-05,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-04,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-05,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-06,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [213]:
df_proc.reset_index()['index'].dtype

dtype('O')

In [ ]:
pd.options.display.float_format = '{:.2f}'.format

In [246]:
pd.to_datetime(df_proc['date'].replace(['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez'],
                                       ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], regex=True), format='%Y-%b').dt.strftime('%Y-%m')

0      2008-01
1      2008-02
2      2008-03
3      2008-04
4      2008-05
        ...   
147    2020-04
148    2020-05
149    2020-06
150    2020-07
151    2020-08
Name: date, Length: 152, dtype: object

In [294]:
df_proc

uf,date,RO,AC,AM,RR,PA,AP,TO,MA,PI,...,RJ,SP,PR,SC,RS,MS,MT,GO,DF,date_str
0,2008-01-01,1388528.39,902416.0,4735529.42,657889.53,18864744.11,1120973.34,3441592.69,10723506.39,7985142.43,...,23025258.52,91678473.48,38266347.46,16504913.45,30931325.46,5377392.11,4795299.01,15310637.29,4745919.19,2008-01
1,2008-02-01,2931283.42,1497206.26,7118990.57,777939.31,19553758.2,1268740.33,4985688.85,13559679.41,10079928.25,...,28394310.93,148677526.82,55384670.52,25651429.11,57363420.01,7002472.61,8684882.23,22258668.68,7632480.67,2008-02
2,2008-03-01,1541682.52,1794028.48,8196635.49,718688.03,21937342.7,974651.24,6205612.6,13985142.36,10298372.45,...,44567062.94,156087269.23,53801318.85,26743524.89,51547794.07,8423933.19,7886243.14,21442515.75,9647203.26,2008-03
3,2008-04-01,1525314.96,1730469.42,8259378.42,839994.39,20842829.69,1141137.32,4984482.09,14809416.46,10775976.33,...,44054604.82,161227125.43,52460306.69,27181488.26,56673338.49,8797487.68,8794109.79,20921212.45,11098732.99,2008-04
4,2008-05-01,1645953.84,1819443.92,7831399.9,862347.96,23249952.88,866346.18,5511002.46,14505727.38,10398360.88,...,44911021.15,187119397.64,54419620.71,26845308.18,48397733.07,9770421.8,9231645.4,21615970.67,12891719.72,2008-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,2020-04-01,7273843.63,2286642.6,13509579.29,2868820.96,28217249.66,1705873.64,6390358.03,24792925.68,12224393.97,...,71403516.21,251320040.41,94599403.51,51544849.24,83000318.02,17410262.53,12546999.04,30896577.34,21536505.16,2020-04
148,2020-05-01,7652120.28,2896340.24,19658585.46,2752228.26,32617541.05,2065055.35,6697839.73,30512523.7,12658280.69,...,85094073.33,272486957.7,107503816.77,55111547.06,82381774.95,17913161.92,13604225.34,34305180.56,19058243.91,2020-05
149,2020-06-01,8725530.68,3045245.8,20897542.93,3401864.29,41625331.28,2009912.57,7384806.71,33099356.79,14547134.38,...,91507444.94,306425499.3,96977288.37,54457911.56,86051002.13,17331445.92,13151005.63,35701473.15,22704335.27,2020-06
150,2020-07-01,11152107.08,3915193.2,19346450.18,3015488.3,38438873.79,3249565.87,9131707.6,32936503.37,17871598.61,...,98722687.54,366721243.71,109658360.43,63467828.84,100531035.83,18755557.92,16371639.49,39405050.36,25269773.34,2020-07
